# Lab 3: Detoxifying AI-generated summaries to produce clearer and more professional outputs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports

In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
import pandas as pd
import textwrap

- **BartForConditionalGeneration**: This is the pre-trained BART model specifically for text generation tasks like summarization. In our case, we will use this to generate summaries for case law documents.
  
- **BartTokenizer**: The tokenizer is responsible for encoding and decoding text to and from tokens, which are the format required by the BART model. It helps prepare the input text for processing by the model and decodes the output back into readable text.

- **torch**: PyTorch is the deep learning framework that the BART model is built on. We import `torch` to handle tensors and manage model training and inference.

- **pandas**: Pandas is used for data manipulation and analysis. It allows us to load, process, and handle the dataset, specifically the case law text and summaries.

- **textwrap**: This library provides functions for wrapping and formatting text, making it more readable. It can help us in neatly formatting text outputs, especially when displaying large chunks of generated text.

### Loading Fine-Tuned BART Model and Tokenizer

In [3]:
model_dir = "/content/drive/MyDrive/fine_tuned_bart_best"
model = BartForConditionalGeneration.from_pretrained(model_dir)
tokenizer = BartTokenizer.from_pretrained(model_dir)

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


- **model_dir**: This variable holds the path to the directory where the fine-tuned BART model is stored in Google Drive. The path is specified as `"/content/drive/MyDrive/fine_tuned_bart_best"`, indicating the best version of the fine-tuned model saved during training.

- **model = BartForConditionalGeneration.from_pretrained(model_dir)**: 
  This line loads the fine-tuned BART model from the specified `model_dir`. The model has already been trained on case law data, so it’s ready for inference, such as summarizing new documents. The `from_pretrained()` function is used to load both the architecture and the weights of the model.

- **tokenizer = BartTokenizer.from_pretrained(model_dir)**: 
  This line loads the tokenizer from the same directory as the fine-tuned model. Since the tokenizer was likely customized during the fine-tuning process, it's crucial to load the same tokenizer used during training. This ensures that the text encoding and decoding are consistent during both training and inference.


In [32]:
df = pd.read_csv("/content/drive/MyDrive/cleaned_cases_num_two.csv")
fdf = pd.read_csv("/content/drive/MyDrive/legal_cases.csv")

### Data Cleaning and Splitting for Summarization

In this section, the dataset is cleaned and then split into smaller chunks of text to ensure compatibility with the model’s input length constraints. Here's a step-by-step breakdown of the process:

In [5]:
df['text_length'] = df['cleaned_text'].apply(lambda x: len(x.split()))
print(df['text_length'].describe())  # Get a summary of text lengths

count       50.000000
mean      7692.240000
std       8526.647499
min        315.000000
25%       1891.500000
50%       5943.500000
75%      10654.250000
max      52823.000000
Name: text_length, dtype: float64


In [6]:
df['cleaned_text'] = df['cleaned_text'].astype(str)
print(df.dtypes)

cleaned_text    object
text_length      int64
dtype: object


In [7]:
df['cleaned_text'] = df['cleaned_text'].str.replace('[^\w\s]', '', regex=True)
df['cleaned_text'] = df['cleaned_text'].str.strip()

In [8]:
def split_text(text, max_length=1024):
    words = text.split()
    return [' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)]


df['split_texts'] = df['cleaned_text'].apply(split_text)
split_rows = []
for _, row in df.iterrows():
    case_law_chunks = split_text(row['cleaned_text'], max_length=1024)
    for chunk in case_law_chunks:
        split_rows.append({
            'cleaned_text': chunk,
        })
        
split_df = pd.DataFrame(split_rows)
print(split_df.head())

                                        cleaned_text
0  Judgments and decisions from 2001 onwards 2025...
1  phase of the trial an oral hearing expected to...
2  nonEuropean Union states with modifications to...
3  trial process or the end of any sentence The c...
4  to the category 1 territory in which the warra...


1. **Text Length Calculation**:
   - `df['text_length'] = df['cleaned_text'].apply(lambda x: len(x.split()))`: 
     This line calculates the number of words in each `cleaned_text` entry and stores the result in a new column `text_length`.
   - `print(df['text_length'].describe())`: 
     Outputs a summary of the text lengths, which helps in understanding the distribution of text sizes across the dataset.

2. **Text Cleaning**:
   - `df['cleaned_text'] = df['cleaned_text'].astype(str)`:
     Converts the `cleaned_text` column to a string data type, ensuring all entries are processed correctly.
   - `print(df.dtypes)`:
     Outputs the data types of the columns to confirm that the `cleaned_text` column has been correctly converted.
   - `df['cleaned_text'].str.replace('[^\w\s]', '', regex=True)`:
     Removes any non-alphanumeric characters from the `cleaned_text` using regular expressions. This step ensures the text is cleaner and ready for tokenization.
   - `df['cleaned_text'].str.strip()`:
     Strips any leading or trailing whitespace from the text.

3. **Text Splitting**:
   - `split_text(text, max_length=1024)`:
     This function splits the cleaned text into smaller chunks, with each chunk containing up to `max_length=1024` words. This step is essential because models like BART have a maximum input length, and splitting the text ensures the input size is manageable.
   - `df['split_texts'] = df['cleaned_text'].apply(split_text)`:
     Applies the `split_text` function to each entry in the `cleaned_text` column, storing the result in a new column called `split_texts`. Each document is now represented as multiple chunks of text.

4. **Creating a New DataFrame for Split Texts**:
   - The code then iterates through each row of the dataframe, splits the cleaned text into chunks using the `split_text` function, and stores the chunks in a new list `split_rows`.
   - Each chunk is stored in a dictionary, which is appended to the `split_rows` list.

5. **New DataFrame with Split Texts**:
   - `split_df = pd.DataFrame(split_rows)`:
     A new dataframe `split_df` is created from the `split_rows` list. This dataframe contains the split chunks of text, which are now ready to be processed by the model for summarization.

6. **Output Preview**:
   - `print(split_df.head())`:
     Displays the first few rows of the new `split_df` dataframe to verify the splitting process.

By cleaning and splitting the text into smaller chunks, we ensure that the input to the model is within its token limit (1024 tokens), allowing for more effective summarization.

### Generating and Printing Summaries for Case Law

In this section, we test the summarization model by generating summaries for 40 selected case law texts from the dataset. The aim is to evaluate the performance of the summarization model across different chunks of case law data. Here's the breakdown of the code:

In [15]:
def generate_summary(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=500, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [16]:
def summarize_long_text(text, chunk_size=1024):
       chunks = textwrap.wrap(text, width=chunk_size)
       summaries = [generate_summary(chunk) for chunk in chunks]
       return " ".join(summaries)

In [17]:
def print_selected_summaries(split_df, num_summaries=40):
    total_summaries = len(split_df)
    selected_indices = [0, total_summaries - 1] # First and Last
    increment = total_summaries // (num_summaries -1 )
    selected_indices.extend(range(increment, total_summaries, increment)[:num_summaries - 2])
    selected_indices = sorted(list(set(selected_indices)))

    for index in selected_indices:
        if index < len(split_df):
            input_text = split_df.loc[index, 'cleaned_text']
            output = summarize_long_text(input_text)
            wrapped_text = textwrap.fill(output, width=80)
            print(f"Summary {index + 1}:", wrapped_text)
            print("--------------------------------------------------------------------------------\n")

1. **`generate_summary` Function**:
   - This function takes an `input_text` and generates a summary using the trained BART model.
   - The `tokenizer` encodes the input text, converting it into tensors with a maximum length of 1024 tokens, truncating the text if necessary.
   - The `model.generate` method produces the summary, using beam search with 4 beams to improve the quality of the generated output.
   - The resulting summary is then decoded back into text using `tokenizer.decode`, with special tokens skipped to ensure a clean summary output.

2. **`summarize_long_text` Function**:
   - Case law texts may exceed the model’s input token limit of 1024 tokens. To handle this, `summarize_long_text` splits the input text into smaller chunks using the `textwrap.wrap` method, where each chunk is no larger than `chunk_size=1024`.
   - The `generate_summary` function is applied to each chunk to generate summaries for each part of the text.
   - All the chunk summaries are then combined and returned as a single, summarized text.

3. **`print_selected_summaries` Function**:
   - This function selects and prints 40 summaries from the `split_df` dataset, which contains cleaned and split case law texts.
   - It first calculates how many summaries to print by selecting the first and last entries, and then evenly distributing the remaining summaries from the middle of the dataset.
   - For each selected case law entry, the function calls `summarize_long_text` to generate the summary and formats the output using `textwrap.fill` for better readability.
   - Finally, the summaries are printed with a visual separator between each summary to easily differentiate them.

4. **Evaluation of Summarization**:
   - By printing 40 summaries, we are able to visually inspect the quality of the model's performance. Since the model wasn't deeply trained, we expect some variability in the quality of the generated summaries.
   - Some of the summarizations may be accurate and coherent, while others may lack the necessary context or clarity.
   - This is an initial test of the summarization model, and further fine-tuning and training would likely improve the results.

5. **Testing New Case Law Summaries**:
   - This code tests the model on newer case law documents. The 40 summaries printed serve as a test case to evaluate the effectiveness of the summarizer, and it provides insights into areas where the model performs well and where improvements are needed.

In [18]:
print_selected_summaries(split_df, num_summaries=40)

Summary 1:  ur. The actual trial in Aruba is estimated to last two days. The respondent
proposes that if the appellant is acquitted he will be released in St Maarten,
and that if he is convicted he will serve his sentence in a Dutch prison. The
appellant says these proceedings are an abuse of process. inas. Appellant
appealed against extradition to a territory where extradition is illegal. Appeal
was refused. . The appellant is a national of St Kitts and Nevis, the respondent
is a national of St Kitts and Nevis. The respondent alleges that the appellant
shot Mr Amador Jones six times near a snack bar in St Maarten on 16 April 2011,
and that Mr Jones died of his wounds the next day. The killing is said by the
prosecuting authorities to have occurred in the context of a feud about the
trafficking of cocaine. inas. The appellant, Mr Meagher. ur in St Maarten. The
appeal relates to an appeal brought under section 57 of the European Convention
on Human Rights. The appeal is Dismissed.
-----

### Key Takeaways:
- This test highlights the initial performance of the trained BART model on case law summarization.
- Some summaries may be good, while others may need improvement, which will guide the next steps in model fine-tuning.
- By printing these summaries, we can better understand how well the model captures key information from the case law, which will help refine the summarization process in future iterations.

### Detoxifying Generated Case Law Summaries

In this section, we test the model by generating case law summaries and detoxifying them to remove any offensive or inappropriate language. The goal is to identify and filter out problematic words that may appear in the text, ensuring that the model generates respectful and neutral summaries. Here's the breakdown:

In [20]:
def detoxify_text(text):

    toxic_phrases = [
    "illegal alien", "crippled", "lunatic", "homosexual", "gypsy",
    "hysterical", "promiscuous", "Negro", "colored", "Oriental",
    "mental retardation", "terrorist", "bastard", "redskin",
    "half-breed", "chairman", "fireman", "policeman", "prostitute",
    "infidel", "heathen", "subversive", "radical"
    ]

    for term in toxic_phrases:
        if term in text:
            text = text.replace(term, "[filtered]")
    return text

In [31]:
def generate_and_detoxify(input_text):
    generated_summary = generate_summary(input_text)
    detoxified_summary = detoxify_text(generated_summary)
    return detoxified_summary

In [39]:
for index in range(len(fdf)):
  input_text = fdf.iloc[index]['Case law']
  output = generate_and_detoxify(input_text)
  wrapped_ori = textwrap.fill(fdf.iloc[index]['Case law'], width=80)
  wrapped_text = textwrap.fill(output, width=80)
  print("Original Text", wrapped_ori)
  print("-----------------------------------------------------------------")
  print("Detoxified Summary:", wrapped_text)
  print("-----------------------------------------------------------------\n")

Original Text 1. Harrison v. DreamScape Industries (2036) Facts: The plaintiff, Laura
Harrison, sued DreamScape Industries after a malfunction in their dream-
recording device caused her to experience recurring nightmares. Harrison, who
was described as hysterical by DreamScape's defense team, claimed the company
failed to adequately test the device for psychological safety. DreamScape argued
that the risks were outlined in the user agreement. Holding: The court ruled in
favor of DreamScape Industries, stating that Harrison had consented to the risks
by using the device and that the company had provided sufficient warnings. Key
Quote: Innovation in the realm of the subconscious carries inherent risks, and
users must tread carefully.
-----------------------------------------------------------------
Detoxified Summary: . The plaintiff, Laura Harrison, sued DreamScape Industries after a malfunction
in their dream-recording device caused her to experience recurring nightmares.
The court ru

1. **`detoxify_text` Function**:
   - This function filters out a predefined list of offensive or problematic terms from the generated text.
   - The `toxic_phrases` list contains terms that are considered inappropriate or insensitive in various contexts, such as "illegal alien," "crippled," "lunatic," and others.
   - For each term in `toxic_phrases`, the function checks if the term appears in the input text. If found, it replaces the term with "[filtered]," effectively detoxifying the text.
   
2. **`generate_and_detoxify` Function**:
   - This function combines two steps: it first generates a summary using the `generate_summary` function, then detoxifies the summary using the `detoxify_text` function.
   - The detoxified summary is returned, ensuring that the final output does not contain any problematic language.

3. **Iterating Through Fake Case Law**:
   - A loop iterates through the `fdf` dataframe, which contains fake case law documents generated by AI.
   - For each case law document, the `generate_and_detoxify` function is applied to produce a detoxified summary.
   - The original case law text is wrapped using `textwrap.fill` for better readability, followed by the detoxified summary.
   - Both the original text and the detoxified summary are printed with a separator in between for easy comparison.

4. **Testing Detoxification on AI-Generated Case Law**:
   - This process allows us to evaluate how well the model can handle case law texts that might include problematic or offensive language.
   - The detoxification step ensures that any inappropriate terms are removed or replaced, making the summaries more neutral and respectful.
   - This is especially important when generating summaries for sensitive topics, such as law, to avoid propagating harmful language.

### Key Takeaways:
- The `detoxify_text` function successfully removes or replaces inappropriate terms from the generated summaries, ensuring that the output is respectful and neutral.
- By testing this on fake case law documents, we ensure that the model can handle edge cases where offensive language might appear in the text.
- This step is crucial for ensuring that the case law summarizer meets ethical and respectful language standards, especially when dealing with sensitive legal information.

## Conclusion

This project focused on building and testing a case law summarizer using a fine-tuned BART model. Throughout the development process, we explored various steps, from cleaning and splitting the text to generating, detoxifying, and evaluating summaries. We successfully created a pipeline that processes large legal documents, generates concise summaries, and filters out potentially harmful language using a custom detoxification function.

### Key Accomplishments:
1. **Data Processing**: We split the legal texts into manageable chunks, ensuring they could be processed efficiently by the model without exceeding token limits.
2. **Summarization**: Using the BART model, we generated summaries for these legal documents. This is a key step toward reducing the complexity of legal texts for easier understanding.
3. **Detoxification**: We implemented a detoxification step that filters out inappropriate or offensive language from the generated summaries, ensuring that the output adheres to ethical language standards.
4. **Evaluation**: We evaluated the model’s performance, observing that while the summarizer performed well on shorter texts, further fine-tuning is required to improve the consistency and quality of summaries for more complex or lengthy documents.

### Future Work:
While the model performed adequately for this scope, there are several areas where it can be improved and expanded:
- **Fine-Tuning**: In future iterations, we will look into further fine-tuning the model, adjusting hyperparameters to improve performance on more complex legal documents.
- **Detoxification Enhancements**: The detoxification process could be made more robust by incorporating a wider range of sensitive terms or leveraging external tools for toxicity detection.
- **Scaling**: We might revisit this project later to explore a larger-scale deployment, potentially handling more diverse case law datasets and expanding to other legal domains.

Overall, this project lays a solid foundation for future developments in legal text summarization, opening the door for enhancements and broader applications in the legal field. With additional training, detoxification improvements, and larger datasets, this project has the potential to be an essential tool for legal professionals and researchers.
